# Infer GRN using Scenic

### Necessary Imports

This will be using a package called [pyscenic](https://github.com/aertslab/pySCENIC) which uses the Arboreto algorithm for network inference

Dask is used for distributed computing.

When installing Pyscenic, as of Today (June 10, 2021), there is an issue with the dask. Check out the issue I filed [here](https://github.com/aertslab/pySCENIC/issues/295) The solution: 

#### I re-install pyscenic with version 0.11.1, and downgrade dask==2.30.0 and distributed==2.30.0, which works for me.

In [1]:
import os
import glob
import pickle
import pandas as pd
import numpy as np

from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

import seaborn as sns

import pickle

/gpfs/home/hzaki1/celltypefromgrn/env/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 6103. The TBB threading layer is disabled.
  warnings.warn(problem)


### Define some routes

Here, we will define some routes. They are constants we will use for simplicity.

The structure of the folders:
```
data/
├─ resources/
│  ├─ GSE60361_C1-3005-Expression.txt
│  ├─ metadata.txt
│  ├─ mm_mgi_tfs.txt
│  ├─ motifs-v9-nr.mgi-m0.001-o0.0.tbl
├─ databases/
│  ├─ mm9-500bp-upstream-10species.mc9nr.feather
│  ├─ mm9-500bp-upstream-7species.mc9nr.feather
│  ├─ mm9-tss-centered-10kb-10species.mc9nr.feather
│  ├─ mm9-tss-centered-10kb-7species.mc9nr.feather
│  ├─ mm9-tss-centered-5kb-10species.mc9nr.feather
│  ├─ mm9-tss-centered-5kb-7species.mc9nr.feather
├─ obj/
```

Download the databases and motif annotations [here](https://resources.aertslab.org/cistarget/)

Download the Expression data [here](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE60361)

Download the metadata [here](http://linnarssonlab.org/cortex/)

```obj/``` will be used to store some pickled data

In [2]:
DATA_FOLDER="/gpfs/data/rsingh47/hzaki1/data"
RESOURCES_FOLDER="/gpfs/data/rsingh47/hzaki1/data/resources"
DATABASE_FOLDER = "/gpfs/data/rsingh47/hzaki1/data/databases"
METADATA_FNAME = os.path.join(RESOURCES_FOLDER, 'metadata.txt')
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*.mc9nr.feather")
MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")
MM_TFS_FNAME = os.path.join(RESOURCES_FOLDER, 'mm_mgi_tfs.txt')
SC_EXP_FNAME = os.path.join(RESOURCES_FOLDER, "GSE60361_C1-3005-Expression.txt")
REGULONS_FNAME = os.path.join(DATA_FOLDER, "regulons.p")
MOTIFS_FNAME = os.path.join(DATA_FOLDER, "motifs.csv")

## Load up expression matrix

In [3]:
ex_matrix = pd.read_csv(SC_EXP_FNAME, sep='\t', header=0, index_col=0).T
ex_matrix.shape

(3005, 19972)

In [4]:
ex_matrix.head()

cell_id,Tspan12,Tshz1,Fnbp1l,Adamts15,Cldn12,Rxfp1,2310042E22Rik,Sema3c,Jam2,Apbb1ip,...,Gm20826_loc1,Gm20826_loc2,Gm20877_loc2,Gm20877_loc1,Gm20865_loc4,Gm20738_loc4,Gm20738_loc6,Gm21943_loc1,Gm21943_loc3,Gm20738_loc3
1772071015_C02,0,3,3,0,1,0,0,11,1,0,...,0,0,0,0,0,0,0,0,0,0
1772071017_G12,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1772071017_A05,0,0,6,0,1,0,2,25,1,0,...,0,0,0,0,0,0,0,0,0,0
1772071014_B06,3,2,4,0,0,0,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1772067065_H06,0,2,1,0,0,0,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
tf_names = load_tf_names(MM_TFS_FNAME)

## Load up the ranking databases

In [6]:
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

[FeatherRankingDatabase(name="mm9-500bp-upstream-7species.mc9nr"),
 FeatherRankingDatabase(name="mm9-tss-centered-5kb-7species.mc9nr"),
 FeatherRankingDatabase(name="mm9-tss-centered-10kb-7species.mc9nr"),
 FeatherRankingDatabase(name="mm9-tss-centered-10kb-10species.mc9nr"),
 FeatherRankingDatabase(name="mm9-500bp-upstream-10species.mc9nr"),
 FeatherRankingDatabase(name="mm9-tss-centered-5kb-10species.mc9nr")]

## Now, the fun part. Let's infer a GRN

This may take a bit, so grab a snack or a cup of tea

In [7]:
adjacencies = grnboost2(ex_matrix, tf_names=tf_names, verbose=True)

preparing dask client
parsing input
creating dask graph
3 partitions
computing dask graph


Task exception was never retrieved
future: <Task finished coro=<connect.<locals>._() done, defined at /gpfs/home/hzaki1/celltypefromgrn/env/lib/python3.7/site-packages/distributed/comm/core.py:288> exception=CommClosedError()>
Traceback (most recent call last):
  File "/gpfs/home/hzaki1/celltypefromgrn/env/lib/python3.7/site-packages/distributed/comm/core.py", line 298, in _
    write = await asyncio.wait_for(comm.write(local_info), 1)
  File "/gpfs/runtime/opt/python/3.7.4/lib/python3.7/asyncio/tasks.py", line 435, in wait_for
    await waiter
concurrent.futures._base.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/gpfs/home/hzaki1/celltypefromgrn/env/lib/python3.7/site-packages/distributed/comm/core.py", line 304, in _
    raise CommClosedError() from e
distributed.comm.core.CommClosedError


shutting down client and local cluster


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


finished


KeyboardInterrupt: 

In [7]:
modules = list(modules_from_adjacencies(adjacencies, ex_matrix))


2021-06-10 17:23:35,455 - pyscenic.utils - INFO - Calculating Pearson correlations.

2021-06-10 17:23:37,167 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].

2021-06-10 17:24:24,261 - pyscenic.utils - INFO - Creating modules.


In [8]:
def save_obj(obj, name):
    with open('data/obj/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_obj(name):
    with open('data/obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [9]:
save_obj(adjacencies, "adjacencies")

In [10]:
save_obj(modules, "modules")

In [11]:
adjacencies.head(40)

,TF,target,importance
65,Gm14305,OTTMUSG00000016609_loc3,428.929097
65,Gm14305,OTTMUSG00000016609_loc4,318.199592
267,Dab2,Mrc1,148.336123
868,Rpl35,Rpl32,148.073154
239,Cers2,Plp1,146.441894
82,Mef2c,Camk2n1,141.429825
868,Rpl35,Rps29,134.163491
267,Dab2,Pf4,134.085725
868,Rpl35,Rps20,132.402329
267,Dab2,Cbr2,130.958951


In [16]:
adjacencies.to_csv('data/adjacencies.tsv', sep='\t', header=True, index=False)

## Let's just take a look at the meta data

In [13]:
df_metadata = pd.read_csv(METADATA_FNAME,  sep='\t', index_col=1, nrows=9).drop(columns=['Unnamed: 0']).T.reset_index() #.drop(columns=['index', 'group #'])
df_metadata.columns.name = ''

In [14]:
df_metadata.age = df_metadata.age.astype(int)

In [15]:
df_metadata.head()

,index,group #,total mRNA mol,well,sex,age,diameter,cell_id,level1class,level2class
0,sscortex,1,21580,11,1,21,0,1772071015_C02,interneurons,Int10
1,sscortex.1,1,21748,95,-1,20,9.56,1772071017_G12,interneurons,Int10
2,sscortex.2,1,31642,33,-1,20,11.1,1772071017_A05,interneurons,Int6
3,sscortex.3,1,32916,42,1,21,11.7,1772071014_B06,interneurons,Int10
4,sscortex.4,1,21531,48,1,25,11,1772067065_H06,interneurons,Int9
